In [1]:
! pip install requests

In [2]:
! pip install beautifulsoup4

In [3]:
from bs4 import BeautifulSoup
import requests

page_link = 'http://www.espn.com/nfl/story/_/id/25824920/angry-new-orleans-saints-fans-file-lawsuits-put-billboards'

page_response = requests.get(page_link, timeout = 5)

page_content = BeautifulSoup(page_response.content, "html.parser")

textContent = []
for i in range(0, 26):
    paragraphs = page_content.find_all("p")[i].text
    textContent.append(paragraphs)
    
for i in textContent:
    print (i)

ConnectionError: HTTPConnectionPool(host='www.espn.com', port=80): Max retries exceeded with url: /nfl/story/_/id/25824920/angry-new-orleans-saints-fans-file-lawsuits-put-billboards (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001DAB396AD30>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed',))

In [4]:
from urllib.request import urlopen
import re
 
html_page = urlopen("http://www.espn.com/nfl/team/_/name/buf/buffalo-bills")
soup = BeautifulSoup(html_page)
for link in soup.find_all('a'):
        print(link.get('href'))

URLError: <urlopen error [Errno 11001] getaddrinfo failed>

In [5]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [7]:
def pprint(soup):# pretty print
    print(soup.prettify())

In [11]:
html_page =  requests.get("http://www.espn.com/nfl/team/_/name/mia/miami-dolphins")
soup = BeautifulSoup(html_page.content)

C:\Users\atenk\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\atenk\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [12]:
# we know that all article links are in the 'article' tag
articles = [art for art in soup.find_all('article') if 'data-id' in art.attrs.keys()] # remove extraneous articles
print(len(articles))


25


In [13]:
for art in articles:
    print('---'*30)
    print(art['class'])
    print(art.a['class'])

------------------------------------------------------------------------------------------
['news-feed-item', 'news-feed-story-package']
['story-link']
------------------------------------------------------------------------------------------
['news-feed-item', 'news-feed-story-package']
['story-link']
------------------------------------------------------------------------------------------
['news-feed-item', 'news-feed-story-package']
['story-link']
------------------------------------------------------------------------------------------
['news-feed-item', '']
['story-link']
------------------------------------------------------------------------------------------
['news-feed-item', 'news-now', 'news-feed-shortstop', 'has-media', 'module_bloom_behavior', '']
['btn-social', 'sm', 'icon-font-before', 'icon-facebook-solid-before', 'Shortstop']
------------------------------------------------------------------------------------------
['news-feed-item', 'news-now', 'news-feed-shortstop',

In [14]:
article = articles[0]
for child in article .children:
    print(child.name)
print()
pprint(article)

a
figure
div

<article class="news-feed-item news-feed-story-package" data-id="philadelphia-eagles-27094">
 <a class="story-link" data-id="philadelphia-eagles-27094" data-popup-href="http://espn.com/blog/philadelphia-eagles/post/_/id/27094/jacksonville-bound-a-look-at-nick-foles-market-after-flacco-deal" data-sport="nfl" href="/blog/philadelphia-eagles/post/_/id/27094/jacksonville-bound-a-look-at-nick-foles-market-after-flacco-deal" name="&amp;lpos=nfl:feed:xx:news">
 </a>
 <figure class="feed-item-figure ">
  <div class="img-wrap">
   <a data-mptype="image" data-sport="nfl" href="/blog/philadelphia-eagles/post/_/id/27094/jacksonville-bound-a-look-at-nick-foles-market-after-flacco-deal" name="&amp;lpos=nfl:feed:xx:news">
    <picture>
     <source data-srcset="https://a2.espncdn.com/combiner/i?img=%2Fphoto%2F2019%2F0211%2Fr500550_1296x518_5%2D2.jpg&amp;w=375&amp;h=150&amp;scale=crop&amp;cquality=80&amp;location=origin, https://a2.espncdn.com/combiner/i?img=%2Fphoto%2F2019%2F0211%2Fr500

In [15]:
article_info = {}

#if article['class'] = ['news-feed-item', 'news-feed-story-package']:
    
for child in article.children:
    if child.name == 'a':
        article_info['class'] = child['class'][0]
        article_info['data-id'] = child['data-id']
        article_info['url'] = child['data-popup-href']
        article_info['sport'] = child['data-sport']
    if child.name == 'div':
        for span in child.div.div.children: # should be a timestap and author span tag
            if 'timestamp' in span['class']: # Beautiful soup always makes the class a list (NOT a string)
                article_info['timestamp'] = span.string
            elif 'author' in span['class']:
                article_info['author'] = span.string

article_info

{'author': 'Tim McManus',
 'class': 'story-link',
 'data-id': 'philadelphia-eagles-27094',
 'sport': 'nfl',
 'timestamp': '15h',
 'url': 'http://espn.com/blog/philadelphia-eagles/post/_/id/27094/jacksonville-bound-a-look-at-nick-foles-market-after-flacco-deal'}

In [16]:
def extract_articles(soup, teamname): 
    articles = [art for art in soup.find_all('article') if 'data-id' in art.attrs.keys()]

    articles_list = []
    for article in articles:
        if 'story-link' in article.a['class']: # should be ['news-feed-item', 'news-feed-story-package']
            article_info = {}
            article_info['teamname'] = teamname
            for child in article.children:
                if child.name == 'a':
                    article_info['class'] = child['class'][0]
                    article_info['data-id'] = child['data-id']
                    article_info['url'] = child['data-popup-href']
                    article_info['sport'] = child['data-sport']
                if child.name == 'div':
                    for span in child.div.div.children: # should be a timestap and author span tag
                        if 'timestamp' in span['class']: # Beautiful soup always makes the class a list (NOT a string)
                            article_info['timestamp'] = span.string
                        elif 'author' in span['class']:
                            article_info['author'] = span.string
                            
        articles_list.append(article_info)
    
    # convert list of dictionaries into dataframe
    df = pd.DataFrame(articles_list)
    return df

def get_df_from_teamname_link(link):
    teamname = link.split('/')[-1] # grab top link (ex 'buffalo-bills')
    html_page =  requests.get(link)
    soup = BeautifulSoup(html_page.content, 'lxml')
    df = extract_articles(soup, teamname)
    return df

In [17]:
link = "http://www.espn.com/nfl/team/_/name/mia/miami-dolphins"
df = get_df_from_teamname_link(link)
df

,author,class,data-id,sport,teamname,timestamp,url
0,Tim McManus,story-link,philadelphia-eagles-27094,nfl,miami-dolphins,15h,http://espn.com/blog/philadelphia-eagles/post/...
1,Cameron Wolfe,story-link,miami-dolphins-28698,nfl,miami-dolphins,15h,http://espn.com/blog/miami-dolphins/post/_/id/...
2,Dan Graziano,story-link,25988932,nfl,miami-dolphins,6h,http://www.espn.com/nfl/story/_/id/25988932/we...
3,Cameron Wolfe,story-link,25953625,nfl,miami-dolphins,6d,http://www.espn.com/nfl/story/_/id/25953625/mi...
4,Cameron Wolfe,story-link,25953625,nfl,miami-dolphins,6d,http://www.espn.com/nfl/story/_/id/25953625/mi...
5,Cameron Wolfe,story-link,25953625,nfl,miami-dolphins,6d,http://www.espn.com/nfl/story/_/id/25953625/mi...
6,Cameron Wolfe,story-link,25953625,nfl,miami-dolphins,6d,http://www.espn.com/nfl/story/_/id/25953625/mi...
7,Cameron Wolfe,story-link,25953625,nfl,miami-dolphins,6d,http://www.espn.com/nfl/story/_/id/25953625/mi...
8,Cameron Wolfe,story-link,25938078,nfl,miami-dolphins,8d,http://www.espn.com/nfl/story/_/id/25938078/do...
9,Jeremy Willis,story-link,25932328,nfl,miami-dolphins,9d,http://www.espn.com/nfl/story/_/id/25932328/nf...


In [18]:
df.drop_duplicates('data-id')

,author,class,data-id,sport,teamname,timestamp,url
0,Tim McManus,story-link,philadelphia-eagles-27094,nfl,miami-dolphins,15h,http://espn.com/blog/philadelphia-eagles/post/...
1,Cameron Wolfe,story-link,miami-dolphins-28698,nfl,miami-dolphins,15h,http://espn.com/blog/miami-dolphins/post/_/id/...
2,Dan Graziano,story-link,25988932,nfl,miami-dolphins,6h,http://www.espn.com/nfl/story/_/id/25988932/we...
3,Cameron Wolfe,story-link,25953625,nfl,miami-dolphins,6d,http://www.espn.com/nfl/story/_/id/25953625/mi...
8,Cameron Wolfe,story-link,25938078,nfl,miami-dolphins,8d,http://www.espn.com/nfl/story/_/id/25938078/do...
9,Jeremy Willis,story-link,25932328,nfl,miami-dolphins,9d,http://www.espn.com/nfl/story/_/id/25932328/nf...
10,Cameron Wolfe,story-link,miami-dolphins-28677,nfl,miami-dolphins,9d,http://espn.com/blog/miami-dolphins/post/_/id/...
13,Jason Reid,story-link,158461,nfl,miami-dolphins,9d,https://theundefeated.com/features/dolphins-fl...
14,Cameron Wolfe,story-link,miami-dolphins-28578,nfl,miami-dolphins,10d,http://espn.com/blog/miami-dolphins/post/_/id/...
15,Cameron Wolfe,story-link,25925144,nfl,miami-dolphins,10d,http://www.espn.com/nfl/story/_/id/25925144/do...
